In [1]:
val columns = spark.read.parquet("s3://agimodeltrainer/Clean_Data/ColumnMap.parquet").as[(String, Seq[String])].collect.toMap
val finalTable = spark.read.parquet("s3://agimodeltrainer/Clean_Data/FinalTable.parquet")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1598555940382_0001,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

columns: scala.collection.immutable.Map[String,Seq[String]] = Map(Census_Business_Patterns -> List(FIPS, Year, Maps), Complex_Race -> List(FIPS, Year, One_Race, Only_White, Only_Black, Only_Native, Only_Cherokee, Only_Chippewa, Only_Navajo, Only_Sioux, Only_Asian, Only_Indian, Only_Chinese, Only_Filipino, Only_Japanese, Only_Korean, Only_Vietnamese, Only_Asian_Other, Only_Pacific, Only_Hawaiian, Only_Guamanian, Only_Samoan, Only_Pacific_Other, Only_Other, Twoplus_Races, White_Black, White_Native, White_Asian, Black_Native), Simple_Race -> List(FIPS, Year, White, Black, Native, Asian, Pacific, Other), Hispanic -> List(FIPS, Year, Hispanic, Mexican, Puerto_Rican, Cuban, Hispanic_Other, Not_Hispanic), Educational_Attainment -> List(FIPS, Year, Pop_18_24, Less_HS_Grad_18_24, HS_Grad_18_24, ...finalTable: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 71 more fields]


In [92]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._

def customSummary(table:DataFrame, columns:Seq[String], aggs:Seq[String]) = {
    val newTable = new Array[Array[String]](aggs.size)
    var newSchema = List(StructField("", StringType, false))
    for ((col, i) <- columns.zipWithIndex) {
        newSchema = newSchema :+ StructField(col, StringType, false)
    }
    
    for ((agg, i) <- aggs.zipWithIndex) {
        newTable(i) = new Array[String](columns.size+1)
        newTable(i)(0) = agg
        val flag = !agg.contains('%')
        for ((col, j) <- columns.zipWithIndex) {
            val temp1 = if (flag) {
                table.select(expr(s"${agg}(${col})")).collect.head
            } else {
                val columnArray = table.select(col).orderBy(col).map(_.get(0).toString).collect
                val percent = agg.substring(0, agg.indexOf('%')).toDouble/100.0
                val index = Math.round((columnArray.length-1) * percent).toInt
                Row(columnArray(index))
            }
            newTable(i)(j+1) = try {
                temp1.getInt(0).toString
            } catch {
                case e: ClassCastException => try {
                    temp1.getLong(0).toString
                } catch {
                    case e: ClassCastException => try {
                        f"${temp1.getDouble(0)}%1.2f"
                    } catch {
                        case e: ClassCastException => temp1.getString(0)
                    }
                }
            }
        }
    }
    val t = newTable.toSeq.map(Row(_:_*))
    val someDF = spark.createDataFrame(
        spark.sparkContext.parallelize(t),
        StructType(newSchema)
    )
    someDF.show
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql._
import org.apache.spark.sql.types._
customSummary: (table: org.apache.spark.sql.DataFrame, columns: Seq[String], aggs: Seq[String])Unit


In [70]:
val selected = columns("Simple_Race")//.diff(Seq("FIPS", "Year"))
finalTable.select(selected.head, selected.tail: _*).summary().show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

selected: Seq[String] = List(FIPS, Year, White, Black, Native, Asian, Pacific, Other)
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|summary|              FIPS|              Year|            White|             Black|           Native|             Asian|            Pacific|             Other|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|  count|             21972|             21972|            21972|             21972|            21972|             21972|              21972|             21972|
|   mean| 30393.87233752048|2014.0003640997634|85.61506007646088| 9.718728381576552|2.883783906790462|1.6029127981066897|0.17578281449117472|2.2876661205170272|
| stddev|15153.499264321712|1.9998861772627188| 16.3791392499106|14.657830440780335|8.109682410891063| 3.1925

In [93]:
customSummary(
    finalTable,
    selected, 
    Seq("count", "mean", "stddev", "min", "25%", "50%", "75%", "max")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-------+------+-----+------+-----+-------+-----+
|      |    FIPS|   Year| White|Black|Native|Asian|Pacific|Other|
+------+--------+-------+------+-----+------+-----+-------+-----+
| count|   21972|  21972| 21972|21972| 21972|21972|  21972|21972|
|  mean|30393.87|2014.00| 85.62| 9.72|  2.88| 1.60|   0.18| 2.29|
|stddev|15153.50|   2.00| 16.38|14.66|  8.11| 3.19|   1.02| 3.75|
|   min|   01001|   2011|  3.80| 0.00|  0.00| 0.00|   0.00| 0.00|
|   25%|   18179|   2012|  80.0|  1.0|   0.6|  0.4|    0.0|  0.4|
|   50%|   29177|   2014|  92.4|  2.9|   1.0|  0.8|    0.1|  1.0|
|   75%|   45081|   2016|  97.1| 11.5|   1.9|  1.6|    0.2|  2.5|
|   max|   56045|   2017|100.00|87.20| 96.00|61.70|  34.30|58.80|
+------+--------+-------+------+-----+------+-----+-------+-----+



Exception in thread cell_monitor-93:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3297



In [ ]:
//TODO: Try more built in functions. Write some applicable custom functions. Move on to visualizations.